In [1]:
!python -V

Python 3.11.3


'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd

In [3]:
first = pd.read_parquet(r'C:/Users/Ikram/OneDrive - NOVAIMS/Desktop/MLOps/mlops-zoomcamp-forked/data/green_tripdata_2021-01.parquet')
second = pd.read_parquet(r'C:/Users/Ikram/OneDrive - NOVAIMS/Desktop/MLOps/mlops-zoomcamp-forked/data/green_tripdata_2021-01.parquet')

In [5]:
first.to_csv(r'C:/Users/Ikram/OneDrive - NOVAIMS/Desktop/MLOps/mlops-zoomcamp-forked/data/green_tripdata_2021-01.csv')
second.to_csv(r'C:/Users/Ikram/OneDrive - NOVAIMS/Desktop/MLOps/mlops-zoomcamp-forked/data/green_tripdata_2021-02.csv')

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [53]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [54]:
import mlflow

In [ ]:
# Get a list of all experiments
experiments = mlflow


# Iterate over the experiments and print their IDs and names
for experiment in experiments:
    experiment_id = experiment.experiment_id
    experiment_name = experiment.name
    print(f"Experiment ID: {experiment_id}, Name: {experiment_name}")

In [132]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
# Delete an experiment by name
#mlflow.delete_experiment(experiment_id=9)

mlflow.set_experiment("nyc-taxi-experiment-zied-mlops")



<Experiment: artifact_location=('file:///c:/Users/Ikram/OneDrive - '
 'NOVAIMS/Desktop/MLOps/mlops-zoomcamp-forked/02-experiment-tracking/mlruns/11'), creation_time=1688739948037, experiment_id='11', last_update_time=1688739948037, lifecycle_stage='active', name='nyc-taxi-experiment-zied-mlops', tags={}>

In [56]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [57]:
df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
df_val = read_dataframe('./data/green_tripdata_2021-02.csv')

C:\Users\Ikram\AppData\Local\Temp\ipykernel_11112\3970424726.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
C:\Users\Ikram\AppData\Local\Temp\ipykernel_11112\3970424726.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [58]:
len(df_train), len(df_val)

(73908, 73908)

In [59]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [60]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [61]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [62]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.6995641181989765

In [25]:
# Tracking this experiment :
with mlflow.start_run(run_name="alpha : 0.0001"):
    

    
    # Set the tag and Log the path parameters :
    mlflow.set_tag("Ml Engineer" , "Zied Triki")
    mlflow.log_param("Train_Data_Path" , "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("Validation_Data_Path" , "./data/green_tripdata_2021-02.csv")

    
    alpha = 0.0001
    # Log the Alpha parameter :
    mlflow.log_param("alpha of the lasso Model" , alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    # Log the Metric :
    mlflow.log_metric("rmse of the Lasso Model" , rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

    #with open('models/lin_reg.bin', 'wb') as f_out:
    #    pickle.dump((dv, lr), f_out)
    # Log the resulting Model :
    #mlflow.log_artifact("lin_reg.bin")

In [23]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [76]:
with mlflow.start_run(run_name="Model Versionning"):

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [63]:
import xgboost as xgb

In [64]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [65]:
train = xgb.DMatrix(X_train, label=y_train )
valid = xgb.DMatrix(X_val, label=y_val)

In [72]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [74]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 0, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}


optimal_result= fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=Trials()
)

[19:02:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.67801                         
[1]	validation-rmse:17.43571                         
[2]	validation-rmse:16.29324                         
[3]	validation-rmse:15.24388                         
[4]	validation-rmse:14.28081                         
[5]	validation-rmse:13.39847                         
[6]	validation-rmse:12.59106                         
[7]	validation-rmse:11.85340                         
[8]	validation-rmse:11.18082                         
[9]	validation-rmse:10.56846                         
[10]	validation-rmse:10.01252                        
[11]	validation-rmse:9.50849                         
[12]	validation-rmse:9.05264                         
[13]	validation-rmse:8.64118                         
[14]	validati

In [130]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
# Delete an experiment by name
#mlflow.delete_experiment(experiment_id=9)

mlflow.set_experiment("Zied_MLOps_Engineer_Experiment")

<Experiment: artifact_location=('file:///c:/Users/Ikram/OneDrive - '
 'NOVAIMS/Desktop/MLOps/mlops-zoomcamp-forked/02-experiment-tracking/mlruns/12'), creation_time=1689016137632, experiment_id='12', last_update_time=1689016137632, lifecycle_stage='active', name='Zied_MLOps_Engineer_Experiment', tags={}>

In [43]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split , RandomizedSearchCV
from sklearn.metrics import   f1_score , roc_auc_score , accuracy_score

# Load and split the data
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)


In [46]:
param_grid = {
    "max_depth": [3,4],
    "learning_rate": [0.1,0.2],
    "n_estimators": [100,200]
}

# Create an XGBoost classifier
model = xgb.XGBClassifier()

# Perform hyperparameter optimization using RandomizedSearchCV
search = RandomizedSearchCV(model, param_grid, scoring="accuracy", n_iter=2, cv=10   )
search.fit(X_train, y_train)

# Get the best model from the hyperparameter search
best_model = search.best_estimator_

In [48]:
#model = xgb.XGBClassifier()
#model.fit(X_train, y_train)
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

In [51]:
# Start an MLflow run
with mlflow.start_run( run_name="Hyperparameter opt with Randomized grid search"):

    mlflow.set_tag("Ml Engineer" , "Zied Triki")
    # Log the XGBoost model as an MLflow artifact
    mlflow.sklearn.log_model(model, "xgboost_model")

    # Log the best hyperparameters
    mlflow.log_params(search.best_params_)

    # Log evaluation metric (e.g., f1_score_)
    f1_score_train = f1_score(y_train, y_pred_train, average='weighted')
    mlflow.log_metric("f1_score_train", f1_score_train)
    f1_score_test = f1_score(y_test, y_pred_test, average='weighted')
    mlflow.log_metric("f1_score_test", f1_score_test)

    accuracy_train = accuracy_score(y_train, y_pred_train)
    mlflow.log_metric("accuracy_train", accuracy_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    mlflow.log_metric("accuracy_test", accuracy_test)

c:\Users\Ikram\anaconda3\envs\zied\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Ikram\anaconda3\envs\zied\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
mlflow.xgboost.autolog(disable=True)

In [137]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=50,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact( "models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[20:20:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:18.25785
[1]	validation-rmse:16.67126
[2]	validation-rmse:15.25093
[3]	validation-rmse:13.98160
[4]	validation-rmse:12.84943
[5]	validation-rmse:11.84147
[6]	validation-rmse:10.94722
[7]	validation-rmse:10.15602
[8]	validation-rmse:9.45828
[9]	validation-rmse:8.84458
[10]	validation-rmse:8.30890
[11]	validation-rmse:7.84155
[12]	validation-rmse:7.43373
[13]	validation-rmse:7.08076
[14]	validation-rmse:6.77641
[15]	validation-rmse:6.51604
[16]	validation-rmse:6.29235
[17]	validation-rmse:6.10108
[18]	validation-rmse:5.93755
[19]	validation-rmse:5.79912
[20]	validation-rmse:5.68069
[21]	validation-rmse:5.58065
[22]	validation-rmse:5.49560
[23]	validation-rmse:5.42389
[24]	validation-rmse:5.36068
[25]	validation-rmse:5.30858
[26]	validation-r

c:\Users\Ikram\anaconda3\envs\zied\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Ikram\anaconda3\envs\zied\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [136]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val) 
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2023/07/13 14:02:48 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


2023/07/13 14:02:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Ikram\anaconda3\envs\zied\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."
2023/07/13 14:02:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Ikram\anaconda3\envs\zied\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


AssertionError: c:\Users\Ikram\anaconda3\envs\zied\Lib\distutils\core.py